In [8]:
pip install gradio

In [9]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor#XGBRegressor is used for regression/XGBClassifier is used for classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import gradio as gr

In [10]:
#Load the dataset
df = pd.read_csv("/content/uci-secom.csv")

In [11]:
df.head()

,Time,0,1,2,3,4,5,6,7,8,...,581,582,583,584,585,586,587,588,589,Pass/Fail
0,2008-07-19 11:55:00,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,...,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,-1
1,2008-07-19 12:32:00,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,...,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1
2,2008-07-19 13:17:00,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,...,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1
3,2008-07-19 14:43:00,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,...,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432,-1
4,2008-07-19 15:22:00,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,...,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432,-1


In [12]:
df.isnull().sum()

,0
Time,0
0,6
1,7
2,14
3,14
...,...
586,1
587,1
588,1
589,1


In [15]:
#If you whole data is continous value , take all the mean values in each column and fill the missing values wit hmean value

for col in df.columns:
  if pd.api.types.is_numeric_dtype(df[col]):
    mean_val = df[col].mean()
    df[col].fillna(mean_val,inplace=True)

<ipython-input-15-ca4f5f90bb25>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mean_val,inplace=True)


In [16]:
df.isnull().sum()

,0
Time,0
0,0
1,0
2,0
3,0
...,...
586,0
587,0
588,0
589,0


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Columns: 592 entries, Time to Pass/Fail
dtypes: float64(590), int64(1), object(1)
memory usage: 7.1+ MB


In [50]:
#Split the data in inputs : X and Output : Y
X = df[["1","2","3","4","5"]]
y = df['Pass/Fail']

In [51]:
X

,1,2,3,4,5
0,2564.00,2187.7333,1411.1265,1.3602,100.0
1,2465.14,2230.4222,1463.6606,0.8294,100.0
2,2559.94,2186.4111,1698.0172,1.5102,100.0
3,2479.90,2199.0333,909.7926,1.3204,100.0
4,2502.87,2233.3667,1326.5200,1.5334,100.0
...,...,...,...,...,...
1562,2464.36,2179.7333,3085.3781,1.4843,100.0
1563,2522.55,2198.5667,1124.6595,0.8763,100.0
1564,2379.78,2206.3000,1110.4967,0.8236,100.0
1565,2532.01,2177.0333,1183.7287,1.5726,100.0


In [60]:
y.unique()

array([-1,  1])

In [52]:
# Normalize

#Continous values or numbers we have large number and small number ,if there is a high portion of huge numbers the data can be skewed and cause bias in the model.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [53]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [54]:
#Train the model
model = XGBRegressor()
model.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [55]:
# Gradio Interface
def predict(*inputs):
  inputs = np.array(inputs).reshape(1,-1)
  input_scaled = scaler.transform(inputs)
  pred = model.predict(input_scaled)
  return float(pred[0])


In [56]:
# Select top N features for the UI (for simplicity)
feature_names = df.drop(["Time",'Pass/Fail'],axis=1).columns[:5]

In [57]:
feature_names

Index(['0', '1', '2', '3', '4'], dtype='object')

In [61]:
#create an input component
inputs = [gr.Number(label = feat) for feat in feature_names]

demo = gr.Interface(fn=predict,inputs = inputs,outputs="number",title = "Yeild Prediction")

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://595ebb9cf71346dacb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
